In [1]:
import numpy as np
import pandas as pd
import sys

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
# from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.


In [ ]:
# df = pd.read_csv('/storage/BW2/data/simpsons_dataset.csv')
# df = df.dropna().reset_index(drop=True)
# # Filter Characters by number of apperances -- set to more than once
# df = df[df.groupby('raw_character_text')['raw_character_text'].transform('size') > 1]
# # filter by character... Homer
# homer_opus = df[df['raw_character_text'] == 'Homer Simpson'].reset_index()
# homer_opus = homer_opus.drop(columns='index')
# # Need to see more stuff
# pd.set_option('display.max_colwidth', 600)

In [8]:
# load ascii text and covert to lowercase
filename = 'b.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# For fun... 
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1509125
Total Vocab:  69


In [9]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 64
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1509061


In [10]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# # define the checkpoint
# filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [2]:
# TRYING TO MINIMIZE

# # load ascii text and covert to lowercase
# filename = 'b.txt'
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()

# # create mapping of unique chars to integers
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
# input_shape=(X.shape[1], X.shape[2]) # ==> (64, 1) replaced below...
# Dense(y.shape[1], ...  # ==>  69

In [11]:
# More LAyerS!!! # Define bigger model
model = Sequential()
model.add(LSTM(256, input_shape=(64, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(69, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

# # fit the model
# model.fit(X, y, epochs=10, batch_size=512, callbacks=callbacks_list)

In [12]:
# load the network weights
filename = "weights-improvement-08-1.4693.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
# decode
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [19]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
# print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(80):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
# print("\nDone.")

" ere."
"nice to meet you."
"i had no idea."
"mr. burns, i think w "
e can start the best friends in the world."
"i was the people who can do that th